<a href="https://colab.research.google.com/github/Soufiane07-05/Stocks-Market-Prediction/blob/main/Prediction_Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This projet is an ai building to predict stocks of Apple using 60 day past(PFA)

In [ ]:
#Import the libraries
#fonction de maths
import math
#pandas: analyse de donnees, reader remote date
import pandas_datareader as web
#manipulation des matrices et des tableaux multidimentionels
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt 
plt.style.use("fivethirtyeight")